In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'
                                    ):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# 라이브러리, 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold

from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

## train Data

In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


## Test Data

In [8]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [10]:
test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [11]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


## EDA

In [12]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

- 가족 찾기

In [13]:
train['Group'] = train['PassengerId'].apply(lambda x : str(x[:4]))
test['Group'] = test['PassengerId'].apply(lambda x : str(x[:4]))
train[['PassengerId','Group']]

,PassengerId,Group
0,0001_01,0001
1,0002_01,0002
2,0003_01,0003
3,0003_02,0003
4,0004_01,0004
...,...,...
8688,9276_01,9276
8689,9278_01,9278
8690,9279_01,9279
8691,9280_01,9280


In [14]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Group            object
dtype: object

In [15]:
# `PassengerGroupSize`: How many people are in each `PassangerGroup`?
train['Size'] = train.groupby('Group')['Group'].transform('count')
test['Size'] = test.groupby('Group')['Group'].transform('count')
train[['Group','Size']].sort_values(['Group'])

,Group,Size
0,0001,1
1,0002,1
2,0003,2
3,0003,2
4,0004,1
...,...,...
8688,9276,1
8689,9278,1
8690,9279,1
8691,9280,2


In [16]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Group            object
Size              int64
dtype: object

- Cabin 나누기

In [17]:
train['cd'] = train['Cabin'].str.split('/').str[0]
train['cn'] = train['Cabin'].str.split('/').str[1]
train['cs'] = train['Cabin'].str.split('/').str[2]

test['cd'] = test['Cabin'].str.split('/').str[0]
test['cn'] = test['Cabin'].str.split('/').str[1]
test['cs'] = test['Cabin'].str.split('/').str[2]

- ML을 위해 범주형 데이터 변환

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
label_cols= ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP","cd","cs"]

In [20]:
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test

In [21]:
train, test = label_encoder(train,test,label_cols)

In [22]:
from sklearn.impute import SimpleImputer

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService","Size","cn"]
STRATEGY = 'median'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print("train_data:\n", train.isnull().sum())
print("---")
print("test_data:\n", test.isnull().sum())

train_data:
 PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Group             0
Size              0
cd                0
cn                0
cs                0
dtype: int64
---
test_data:
 PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin            0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
Group            0
Size             0
cd               0
cn               0
cs               0
dtype: int64


In [23]:
imputer_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP","cd","cs"]
STRATEGY = 'most_frequent'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print("train_data:\n", train.isnull().sum())
print("---")
print("test:\n", test.isnull().sum())

train_data:
 PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Group             0
Size              0
cd                0
cn                0
cs                0
dtype: int64
---
test:
 PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin            0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
Group            0
Size             0
cd               0
cn               0
cs               0
dtype: int64


In [24]:
train = train.drop(['Name','Cabin'],axis = 1)
test = test.drop(['Name','Cabin'],axis = 1)

In [25]:
train.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Group           0
Size            0
cd              0
cn              0
cs              0
dtype: int64

---

# 모델

In [26]:
print(train.columns)
print(test.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'Group', 'Size', 'cd', 'cn', 'cs'],
      dtype='object')
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Group',
       'Size', 'cd', 'cn', 'cs'],
      dtype='object')


In [27]:
col = train.drop(['PassengerId','Transported','Group'], axis = 1).columns.tolist()

In [28]:
col

['HomePlanet',
 'CryoSleep',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Size',
 'cd',
 'cn',
 'cs']

In [29]:
TARGET = 'Transported'

In [30]:
X = train[col]
Y = train[TARGET]
X_train, Y_train, X_target, Y_target = train_test_split(X,Y,random_state = 12,test_size = 0.33)

In [31]:
# type_fold_indice = List[Tuple[np.ndarray, np.ndarray]]

idx_fold = []

skf = StratifiedKFold(5, random_state=12, shuffle=True)
for idx_train, idx_valid in skf.split(X = X, y = Y):
    idx_fold.append((idx_train, idx_valid))

In [32]:
lgb_params = {
    'objective' : 'binary',
    'n_estimators' : 200,
    'learning_rate' : 0.1,
    'random_state' : 12,
    'n_jobs' : -1,
    'importance_type' : 'gain',
    'subsample' : 0.9,
    'subsample_freq' : 3,
    'colsample_bytree' : 0.6
}

In [33]:
lgb_scores = []

In [34]:
for fold, (idx_train, idx_valid) in enumerate(idx_fold):
    print(10*"=", f"Fold={fold+1}", 10*"=")
    
    # X_test = test[continuous_features + categorical_features].copy()
    X_train, X_valid = train.iloc[idx_train][col],train.iloc[idx_valid][col]
    Y_train, Y_valid = train.iloc[idx_train][TARGET],train.iloc[idx_valid][TARGET]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, Y_train, verbose = 0)
    
    # 정확도 측정해서 저장
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(Y_valid, preds_valid)
    lgb_scores.append(acc)
    
    print(f"Fold={fold+1}, Accuracy score : {acc :2f}%")
    
print("")
print("Mean Accuracy : ", np.mean(lgb_scores))

========== Fold=1 ==========
Fold=1, Accuracy score : 0.818861%
========== Fold=2 ==========
Fold=2, Accuracy score : 0.802185%
========== Fold=3 ==========
Fold=3, Accuracy score : 0.809086%
========== Fold=4 ==========
Fold=4, Accuracy score : 0.796318%
========== Fold=5 ==========
Fold=5, Accuracy score : 0.806099%

Mean Accuracy :  0.8065097661380991
